In [1]:
import pandas as pd
import altair as alt
import numpy as np
import os
import sys

source_dir = '/Users/matthewdeverna/Documents/Projects/mean_field_quals/src'
sys.path.append(source_dir)


from simulations import run_simulation, get_peak_day, deriv_simple, deriv_with_homophily

In [51]:
def deriv_with_homophily(beta_o, beta_m, sus_o, sus_m, inf_o, inf_m, k, alpha, counts=False, N=None):
    """
    Calculate the *change* in population for all six compartments in the scenario
    where we consider two populations — ordinary and misinformed — (and thus six
    compartments), but no homophily.

    Parameters:
    -----------
    - beta_o (float) : proportion of population infected (ordinary)
    - beta_m (float) : proportion of the population infected (misinformed)
    - sus_o (float)  : proportion of the population susceptible (ordinary)
    - sus_m (float)  : proportion of the population susceptible (misinformed)
    - inf_o (float)  : proportion of the population infected (ordinary)
    - inf_m (float)  : proportion of the population infected (misinformed)
    - k (inf)        : the rate of recovery (i.e., 1 / num days to recover)
    - alpha (float)  : level of homophily. Must fall in range: .5 <= alpha <= 1
    - counts (bool)  : if True, run the simulation based on a number of people
    - N (int)        : size of the population to run with `counts`

    Returns
    -----------
    The *change* in the population for...
    - ds_o : susceptible (ordinary)
    - di_o : infected (ordinary)
    - dr_o : recovered (ordinary)
    - ds_m : susceptible (misinformed)
    - di_m : infected (misinformed)
    - dr_m : recovered (misinformed)
    """
    if not (.5 <= alpha <= 1):
        raise ValueError("`alpha` must fall in the range [.5,1]")
    
    if not counts:
        # Ordinary folks first
        ds_o = -2*beta_o * sus_o * (inf_o*alpha + inf_m*(1-alpha))
        di_o = 2*beta_o * sus_o * (inf_o*alpha + inf_m*(1-alpha)) - k * inf_o
        dr_o = k * inf_o

        # Misinfo folks next
        ds_m = -2*beta_m * sus_m * (inf_o*alpha + inf_m*(1-alpha))
        di_m = 2*beta_m * sus_m * (inf_o*alpha + inf_m*(1-alpha)) - k * inf_m
        dr_m = k * inf_m

    else:
        if N is None:
            raise ValueError("`N` must be set with `counts = True`")

        # Ordinary folks first
        ds_o = -2*beta_o * sus_o * (inf_o*alpha + inf_m*(1-alpha)) / N
        di_o = ((2*beta_o * sus_o * (inf_o*alpha + inf_m*(1-alpha))) / N) - k * inf_o
        dr_o = k * inf_o

        # Misinfo folks next
        ds_m = (-2*beta_m * sus_m * (inf_o*alpha + inf_m*(1-alpha))) / N
        di_m = ((2*beta_m * sus_m * (inf_o*alpha + inf_m*(1-alpha))) / N) - k * inf_m
        dr_m = k * inf_m

    return ds_o, di_o, dr_o, ds_m, di_m, dr_m

In [52]:
x

0.5

In [2]:
# ---- from function input ----

frac_ord = .5
prop_infec = .001
num_days= 100
beta_ord = .3
recovery_days=7
beta_mult = 2

# ---- from function input ----



eps = prop_infec
x = frac_ord

step_size = 1  # step size
all_steps = np.arange(0, num_days, step_size)

### Set initial values ###
S_o = np.zeros(len(all_steps))
S_m = np.zeros(len(all_steps))
I_o = np.zeros(len(all_steps))
I_m = np.zeros(len(all_steps))
R_o = np.zeros(len(all_steps))
R_m = np.zeros(len(all_steps))

# These have unique values for the first time step

if x == 1:
    S_o[0] = x - eps
    I_o[0] = eps
else:
    S_o[0] = x
    S_m[0] = 1 - x - eps
    I_m[0] = eps

# Set recovery rate
k = 1 / recovery_days

# Setting beta values
B_o = beta_ord
B_m = B_o * beta_mult
B_m = 1 if B_m >= 1 else B_m

# Get r0 values
ord_r0 = B_o / k
mis_r0 = B_m / k
weighted_avg_r0 = np.average([ord_r0, mis_r0], weights=[x, 1 - x])

r0s = (ord_r0, mis_r0, weighted_avg_r0)

for t in range(0, len(all_steps) - 1):

    # Calculate the change of each value
    d_s_o, d_i_o, d_r_o, d_s_m, d_i_m, d_r_m = deriv_with_homophily(
        beta_o=B_o,
        beta_m=B_m,
        sus_o=S_o[t],
        sus_m=S_m[t],
        inf_o=I_o[t],
        inf_m=I_m[t],
        k=k,
        alpha=.75
    )

    # Ensure that the total change is zero because individuals should
    # simply be shifting between compartments
    total_change = d_s_o + d_s_m + d_i_o + d_i_m + d_r_o + d_r_m
    assert np.allclose(total_change, 0), f"{total_change}"

    # Set the next value as the current plus it's change
    S_o[t + 1] = S_o[t] + d_s_o
    S_m[t + 1] = S_m[t] + d_s_m
    I_o[t + 1] = I_o[t] + d_i_o
    I_m[t + 1] = I_m[t] + d_i_m
    R_o[t + 1] = R_o[t] + d_r_o
    R_m[t + 1] = R_m[t] + d_r_m

# S_o, S_m, I_o, I_m, R_o, R_m, r0s

In [27]:
print(S_o[0])
print(S_m[0])
print(I_m[0])

0.5
0.499
0.001


In [31]:
I_o[0]

0.0

In [ ]:
((-2*beta_o * sus_o * (inf_o*alpha + inf_m*(1-alpha))) / N) - k * inf_o

In [34]:
-2*B_o * S_o[t]

-0.3

In [38]:
alpha = .75
I_o[t] * alpha

0.0

In [ ]:
ds_o, di_o, dr_o, ds_m, di_m, dr_m

In [31]:
t = 0

deriv_with_homophily(
        beta_o=B_o,
        beta_m=B_m,
        sus_o=S_o[t],
        sus_m=S_m[t],
        inf_o=I_o[t],
        inf_m=I_m[t],
        k=k,
        alpha=1
    )

(-0.0, 0.0, 0.0, -0.0, -0.00014285714285714284, 0.00014285714285714284)

In [4]:
import matplotlib.pyplot as plt

In [14]:
S_o, S_m, I_o, I_m, R_o, R_m, r0s = run_simulation(
    frac_ord = .5,
    prop_infec=.001,
    num_days=100,
    beta_ord=.3,
    recovery_days=7,
    beta_mult=2,
    w_homophily=True,
    alpha = .9,
    counts=False,
    N=None,
)

In [22]:
# Fraction of ordinary people
x = .5

# All beta values
beta = .3

# Less beta values highlighted for one of the later plots
alphas = [0.5, 0.6, 0.7, 0.8, 0.9, 1.]

totals = []      # Total infections
r0s_ = []        # R0 valuyes
peak_days = []   # Date of each peak

# Will store the progression of infections over time, indexed by beta values
infection_flows = dict()


for alpha in alphas:
    
    # Run the simulation based on the input 
    S_o, S_m, I_o, I_m, R_o, R_m, r0s = run_simulation(
        frac_ord=x,
        prop_infec=.001,
        num_days=100,
        beta_ord=beta,
        recovery_days=5,
        beta_mult=2,     # Multiple the current beta value by this to get the misinfo group beta
        w_homophily=True,
        alpha=alpha
    )
    total_ord_inf = max(R_o)
    total_mis_inf = max(R_m)
    
    infection_flows[alpha] = I_o
    
    r0s_.append( (r0s[0], r0s[1], r0s[2], beta) )
    
    totals.append((alpha, total_ord_inf, total_mis_inf, total_ord_inf + total_mis_inf))

In [24]:
totals_df = pd.DataFrame(totals, columns = ['alpha', 'ord_inf', 'mis_inf', 'total'])

In [25]:
totals_df.melt(id_vars='alpha')

,alpha,variable,value
0,0.5,ord_inf,0.356824
1,0.6,ord_inf,0.348444
2,0.7,ord_inf,0.338486
3,0.8,ord_inf,0.325723
4,0.9,ord_inf,0.303217
5,1.0,ord_inf,0.000000
6,0.5,mis_inf,0.461104
7,0.6,mis_inf,0.456011
8,0.7,mis_inf,0.449611
9,0.8,mis_inf,0.440866


In [29]:
alt.Chart(totals_df.melt(id_vars='alpha')).mark_point().encode(
    x = alt.X('alpha:O'),
    y = alt.Y("value"),
    color = alt.Color("variable")
)

alt.Chart(...)

In [30]:
totals_df

,alpha,ord_inf,mis_inf,total
0,0.5,0.356824,0.461104,0.817928
1,0.6,0.348444,0.456011,0.804455
2,0.7,0.338486,0.449611,0.788097
3,0.8,0.325723,0.440866,0.766589
4,0.9,0.303217,0.423963,0.727179
5,1.0,0.000000,0.001000,0.001000


In [9]:
eps

(0.001,)